In [273]:
# import libraries
import pandas as pd
from collections import Counter
from nltk.util import ngrams
import math
import re
import numpy as np
from sklearn.model_selection import train_test_split
import heapq

In [274]:
# load password dataset
df_passwords = pd.read_csv('dataset.csv')
df_passwords.head()

,password,strength,length,class_strength,entropy,crack_time_sec,crack_time
0,bybee,0.088053,5,Very week,11.60964,1.562500e-06,instant
1,n3m0,0.088889,4,Very week,8.00000,1.280000e-07,instant
2,2509,0.088889,4,Very week,8.00000,1.280000e-07,instant
3,4622,0.070443,4,Very week,8.00000,1.280000e-07,instant
4,shrk,0.088889,4,Very week,8.00000,1.280000e-07,instant


In [275]:
dictEncoding = {'very week': 0, 'week': 1, 'average': 2, 'strong': 3, 'very strong': 4}
df_passwords['numeric_class_strength'] = df_passwords['class_strength'].str.lower().map(dictEncoding)
labels = df_passwords['numeric_class_strength']
print(labels)

0        0
1        0
2        0
3        0
4        0
        ..
99995    4
99996    4
99997    4
99998    4
99999    4
Name: numeric_class_strength, Length: 100000, dtype: int64


In [276]:
# generate ngrams from 1 to 5
def generate_ngrams_counts(character_tokens):
    all_ngrams = []
    for i in range(5):
        all_ngrams.extend([''.join(gram) for gram in ngrams(character_tokens, i + 1)])
    return Counter(all_ngrams)

In [277]:
# calculate shannon_entropy given character counts
def shannon_entropy(character_tokens):
    character_counts = Counter(character_tokens)
    total = len(character_tokens)
    return -sum((count / total) * math.log2(count / total) for count in character_counts.values())

In [278]:
# find counts of special characters in the password
def count_special_characters(password):
    # matches any character that is NOT a letter, number, or whitespace
    pattern = r"[^a-zA-Z0-9\s]"  
    return len(re.findall(pattern, password))

# find counts of numbers in the password
def count_numbers(password):
    # matches any digit (0-9)
    pattern = r"\d" 
    return len(re.findall(pattern, password))

# finds counts of uppercase letters in the password
def count_uppercase(password):
    # matches any uppercase letter (A-Z)
    pattern = r"[A-Z]"  
    return len(re.findall(pattern, password))

# finds counts of lowercase letters in the password
def count_lowercase(password):
    # matches any lowercase letter (a-z)
    pattern = r"[a-z]"  
    return len(re.findall(pattern, password))

In [279]:
# calculate tf-idf for the character counts in the password
# tbd ...

In [280]:
# create input dataframe containing passwords and relevant, extracted features
password_inputs = pd.DataFrame()

# add the passwords from the dataset
password_inputs['password'] = df_passwords['password']

# store lengths of each password
password_inputs['length'] = password_inputs['password'].apply(len)

# tokenize input passwords by characters
password_inputs['character_tokens'] = password_inputs['password'].apply(list)

# count upper case letters in password
password_inputs['uppercase_count'] = password_inputs['password'].apply(count_uppercase)

# count lower case letters in password
password_inputs['lowercase_count'] = password_inputs['password'].apply(count_lowercase)

# count numbers in password
password_inputs['numbers_count'] = password_inputs['password'].apply(count_numbers)

# count special character in password
password_inputs['special_character_count'] = password_inputs['password'].apply(count_special_characters)

# find occurrences of each character in the passwords
password_inputs['ngram_occurrences'] = password_inputs['character_tokens'].apply(generate_ngrams_counts)

# find entropy of each passwords (Shannon Entropy)
password_inputs['entropy'] = password_inputs['character_tokens'].apply(shannon_entropy)

password_inputs


,password,length,character_tokens,uppercase_count,lowercase_count,numbers_count,special_character_count,ngram_occurrences,entropy
0,bybee,5,"[b, y, b, e, e]",0,5,0,0,"{'b': 2, 'y': 1, 'e': 2, 'by': 1, 'yb': 1, 'be...",1.521928
1,n3m0,4,"[n, 3, m, 0]",0,2,2,0,"{'n': 1, '3': 1, 'm': 1, '0': 1, 'n3': 1, '3m'...",2.000000
2,2509,4,"[2, 5, 0, 9]",0,0,4,0,"{'2': 1, '5': 1, '0': 1, '9': 1, '25': 1, '50'...",2.000000
3,4622,4,"[4, 6, 2, 2]",0,0,4,0,"{'4': 1, '6': 1, '2': 2, '46': 1, '62': 1, '22...",1.500000
4,shrk,4,"[s, h, r, k]",0,4,0,0,"{'s': 1, 'h': 1, 'r': 1, 'k': 1, 'sh': 1, 'hr'...",2.000000
...,...,...,...,...,...,...,...,...,...
99995,sifelizestasdecirmeloquerras,28,"[s, i, f, e, l, i, z, e, s, t, a, s, d, e, c, ...",0,28,0,0,"{'s': 4, 'i': 3, 'f': 1, 'e': 5, 'l': 2, 'z': ...",3.624519
99996,iwillalwayslovemyboyfriend,26,"[i, w, i, l, l, a, l, w, a, y, s, l, o, v, e, ...",0,26,0,0,"{'i': 3, 'w': 2, 'l': 4, 'a': 2, 'y': 3, 's': ...",3.719295
99997,letsyouupdateyourfunNotesandmore,32,"[l, e, t, s, y, o, u, u, p, d, a, t, e, y, o, ...",1,31,0,0,"{'l': 1, 'e': 4, 't': 3, 's': 2, 'y': 2, 'o': ...",3.726410
99998,chocolatesoeusi912134741,24,"[c, h, o, c, o, l, a, t, e, s, o, e, u, s, i, ...",0,15,9,0,"{'c': 2, 'h': 1, 'o': 3, 'l': 1, 'a': 1, 't': ...",3.855389


In [281]:
# create the input matrix and class vector
X = password_inputs[['length', 'uppercase_count', 'lowercase_count', 'numbers_count', 'special_character_count', 'entropy']].to_numpy()
y = labels.to_numpy().reshape(-1, 1)

# split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [282]:
# calculate accuracy given the true labels and the predictions
def accuracy(y_truth, y_pred):
    correct_pred = 0
    # iterate through the values and check if the labels are the same, update as required
    for y_t, y_p in zip(y_truth, y_pred):
        if y_t == y_p :
            correct_pred += 1
    # find the proportion by dividing the correct predictions by all the predictions
    return correct_pred / len(y_truth)

In [283]:
# softmax function
def softmax(z):
   exp_z = np.exp(z - np.max(z, axis = 1, keepdims = True))
   return exp_z / np.sum(exp_z, axis = 1, keepdims = True)

In [284]:
# predict class using softmax and weights
def softmax_prediction(X, w):
   # add bias terms
   X = np.hstack((np.ones((X.shape[0], 1)), X))
   # return the class with the highest probability as the predicted label
   return np.argmax(softmax(X.dot(w.T)), axis = 1)

In [285]:
# logistic regression function using softmax instead of sigmoid for multinomial classification (gradient descent)
def logistic_regression(X, y, num_classes, iterations, learning_rate):
   # add bias terms
   X = np.hstack((np.ones((X.shape[0], 1)), X))

   # initialize the weights
   w = np.ones((num_classes, X.shape[1]))

   # gradient descent, adjust weights iteratively using the learning rate
   for i in range(iterations):
      # find the predicted 
      class_probabilities = softmax(X.dot(w.T))

      # one hot encoding of labels
      y_one_hot = np.eye(num_classes)[y].reshape(len(y), num_classes)

      # calculate gradient and adjust the weights
      gradient = (class_probabilities - y_one_hot).T.dot(X) / len(y)
      w -= learning_rate * gradient
   return w

In [31]:
# train the logistic regression model
w = logistic_regression(X_train, y_train, 5, 100000, 0.025)

In [32]:
# find the accuracy metrics for each set of data using logistic regression weights
train_predictions = softmax_prediction(X_train, w)
train_accuracy = accuracy(y_train.reshape(1, -1)[0], train_predictions)
print("Logistic Regression Train Accuracy:", train_accuracy)

val_predictions = softmax_prediction(X_val, w)
val_accuracy = accuracy(y_val.reshape(1, -1)[0], val_predictions)
print("Logistic Regression Validation Accuracy:", val_accuracy)

test_predictions = softmax_prediction(X_test, w)
test_accuracy = accuracy(y_test.reshape(1, -1)[0], test_predictions)
print("Logistic Regression Test Accuracy:", test_accuracy)

Logistic Regression Train Accuracy: 0.9047125
Logistic Regression Validation Accuracy: 0.9094
Logistic Regression Test Accuracy: 0.9045


In [286]:
# k nearest neighbors function
def knn(X, num_classes, k, norm_order):
    # initialize list of predictions
    y_pred = []
    for x in X:
        # find the distances to the training data
        distances_with_indexes = []
        # iterate through the training data and find the distances to each point using the normalization order
        for i in range(len(X_train)):
            # store the top k data points that are closest to x
            if len(distances_with_indexes) < k:
                heapq.heappush(distances_with_indexes, (-1 * np.linalg.norm(X_train[i] - x, ord=norm_order), i))
            else:
                heapq.heappushpop(distances_with_indexes, (-1 * np.linalg.norm(X_train[i] - x, ord=norm_order), i))
        # initialize class counts to zero
        class_counts = np.zeros(num_classes)
        # iterate through the k nearest neighbors and find the counts of each label
        for distance, index in distances_with_indexes:
            class_counts[y_train[index]] += 1
        # append the class with the greatest count in the neighbors as the predicted label for this x
        y_pred.append(np.argmax(class_counts))
    return y_pred

In [32]:
# find the accuracy metrics for each set of data using knn
train_predictions = knn(X_train, 5, 20, 2)
train_accuracy = accuracy(y_train.reshape(1, -1)[0], train_predictions)
print("KNN Train Accuracy:", train_accuracy)

KNN Train Accuracy: 0.9876875


In [299]:
# find the accuracy metrics for each set of data using knn
val_predictions = knn(X_val, 5, 20, 5)
val_accuracy = accuracy(y_val.reshape(1, -1)[0], val_predictions)
print("KNN Validation Accuracy:", val_accuracy)

KNN Validation Accuracy: 0.9877


In [300]:
# find the accuracy metrics for each set of data using knn
test_predictions = knn(X_test, 5, 20, 5)
test_accuracy = accuracy(y_test.reshape(1, -1)[0], test_predictions)
print("KNN Test Accuracy:", test_accuracy)

KNN Test Accuracy: 0.9865


In [36]:
# relu activation function
def relu(x):
    return np.maximum(0, x)

# derivative of relu for backprop
def relu_derivative(x):
    return (x > 0).astype(float)

In [443]:
# MLP forward function to pass through inputs and weights/biases to retrieve the outputs of last hidden layer
def forward(X, weights, biases, num_hidden_layers=1, verbose=False):
    # initialize list of layer outputs
    all_layer_outputs = []

    # initial layer's inputs is the original X
    next_layer_inputs = X
    # create num_hidden_layers layers using a for loop
    for layer in range(num_hidden_layers):
        # calculate the inputs to this hidden layer
        layer_logits = next_layer_inputs.dot(weights[layer]) + biases[layer]
        print(layer_logits)
        # find the outputs of the this hidden layer using the activation function
        layer_outputs = relu(layer_logits)
        # save this layer's outputs as the next layer's inputs
        next_layer_inputs = layer_outputs.copy()
        # add the outputs to the cummulative list
        print(weights[layer].shape)
        print(layer_outputs.shape)
        all_layer_outputs.append(layer_outputs.copy())

    # calculate the inputs to the ouput layer
    output_layer_logits = next_layer_inputs.dot(weights[-1]) + biases[-1]
    print(output_layer_logits)
    # find the outputs of the output layer using the activation function
    output_layer_outputs = softmax(output_layer_logits)
    # add the output layer outputs to the cummulative list
    all_layer_outputs.append(output_layer_outputs.copy())

    if verbose:
        print("\nforward:\n", all_layer_outputs[-1])

    # return list of outputs of all layers to calculate the error using backprop
    return all_layer_outputs

In [436]:
# MLP backward function to calculate the error using backpropagation
def backward(X, y, num_hidden_layers, all_layer_outputs, weights, biases, learning_rate, verbose=False):
    # find the number of samples
    samples = y.shape[0]

    # total layers in MLP
    total_layers = num_hidden_layers + 1

    # initialize gradients of each layer's weights and biases as they will be calculated in reverse
    weights_gradient = [None] * total_layers
    biases_gradient = [None] * total_layers

    # calculate the initial error as the difference between predictions and true labels (one-hot encoded y data)
    error = all_layer_outputs[-1] - y
    
    # calculate the gradient of each layer's weights and bias term by using the derivative of loss function 
    for layer in range(num_hidden_layers, 0, -1):
        previous_layer_outputs = all_layer_outputs[layer - 1]
        weights_gradient[layer] = previous_layer_outputs.T.dot(error) / samples
        biases_gradient[layer] = np.sum(error, axis=0, keepdims=True) / samples
        # backpropogate error from this layer to previous layer
        error = error.dot(weights[layer].T) * relu_derivative(all_layer_outputs[layer - 1])
        if verbose:
            print("error", layer, ":", error)

    # calculate gradient of the initial hidden layer's weights and bias term
    weights_gradient[0] = X.T.dot(error) / samples
    biases_gradient[0] = np.sum(error, axis=0, keepdims=True) / samples

    # adjust the weights and bias terms of each layer
    for layer in range(total_layers):
        weights[layer] -= learning_rate * weights_gradient[layer]
        biases[layer] -= learning_rate * biases_gradient[layer]
        
    return weights, biases

In [441]:
# training function for MLP
def MLP_train(X, y, num_classes, num_hidden_layers, num_hidden_nodes_per_layer, learning_rate, epochs):
    # do one-hot encoding of the y labels
    y_one_hot = np.eye(num_classes)[y].reshape(len(y), num_classes)
    # input size is the number of features for each password
    num_features = X.shape[1]

    # initialize weight vectors and bias terms lists (weights and biases for each layer)
    weights = []
    biases = []
    # initial rows in weights is columns in X
    previous_layer_dimension = num_features
    for layer in range(num_hidden_layers):
        weights.append(np.ones((previous_layer_dimension, num_hidden_nodes_per_layer[layer])))
        biases.append(np.ones((1, num_hidden_nodes_per_layer[layer])))
        # update the previous layer dimension
        previous_layer_dimension = num_hidden_nodes_per_layer[layer]

    # add the output layer's weights and bias term
    weights.append(np.ones((previous_layer_dimension, num_classes)))
    biases.append(np.ones((1, num_classes)))

    # iterate through the epochs and adjust the weights/biases to learn
    for epoch in range(epochs):
        # do the forward pass to find the predictions
        # if epoch % 100 == 0:
        #     all_layer_outputs = forward(X, weights, biases, num_hidden_layers=num_hidden_layers, verbose=True)
        #     print("epoch", epoch)
        #     print(all_layer_outputs[-1][:10])
        #     print()
        # else:
        all_layer_outputs = forward(X, weights, biases, num_hidden_layers=num_hidden_layers)
        # do the backward pass to find the error and adjust the weight vectors/bias terms
        if epoch % 100 == 0:
            weights, biases = backward(X, y_one_hot, num_hidden_layers, all_layer_outputs, weights, biases, learning_rate, verbose=False)
            print("epoch", epoch)
            print(weights[:][:10])
            print()
        else:
            weights, biases = backward(X, y_one_hot, num_hidden_layers, all_layer_outputs, weights, biases, learning_rate)
    
    # return the final weights and biases after training
    return weights, biases

In [438]:
# prediction function that takes the argmax of the outputs of the last layer in the MLP
def MLP_predict(X, weights, biases, num_hidden_layers):
    # do the foward pass to find the prediction probability distribution
    all_layer_outputs = forward(X, weights, biases, num_hidden_layers=num_hidden_layers)
    # returns the argmax of the outputs which is the class with the highest probability
    return np.argmax(all_layer_outputs[-1], axis=1)

In [444]:
weights, biases = MLP_train(X_train, y_train, 5, 2, [3, 4], 0.02, 1000)

[[10.5        10.5        10.5       ]
 [78.96621012 78.96621012 78.96621012]
 [38.49922755 38.49922755 38.49922755]
 ...
 [12.52192809 12.52192809 12.52192809]
 [11.         11.         11.        ]
 [32.52164064 32.52164064 32.52164064]]
(6, 3)
(80000, 3)
[[ 32.5         32.5         32.5         32.5       ]
 [237.89863037 237.89863037 237.89863037 237.89863037]
 [116.49768264 116.49768264 116.49768264 116.49768264]
 ...
 [ 38.56578428  38.56578428  38.56578428  38.56578428]
 [ 34.          34.          34.          34.        ]
 [ 98.56492191  98.56492191  98.56492191  98.56492191]]
(3, 4)
(80000, 4)
[[131.         131.         131.         131.         131.        ]
 [952.59452147 952.59452147 952.59452147 952.59452147 952.59452147]
 [466.99073057 466.99073057 466.99073057 466.99073057 466.99073057]
 ...
 [155.26313714 155.26313714 155.26313714 155.26313714 155.26313714]
 [137.         137.         137.         137.         137.        ]
 [395.25968764 395.25968764 395.25968764 39

In [392]:
# find the accuracy metrics for each set of data using MLP
train_predictions = MLP_predict(X_train, weights, biases, num_hidden_layers=1)
train_accuracy = accuracy(y_train.reshape(1, -1)[0], train_predictions)
print("MLP Train Accuracy:", train_accuracy)

val_predictions = MLP_predict(X_val, weights, biases, num_hidden_layers=1)
val_accuracy = accuracy(y_val.reshape(1, -1)[0], val_predictions)
print("MLP Validation Accuracy:", val_accuracy)

test_predictions = MLP_predict(X_test, weights, biases, num_hidden_layers=1)
test_accuracy = accuracy(y_test.reshape(1, -1)[0], test_predictions)
print("MLP Test Accuracy:", test_accuracy)

MLP Train Accuracy: 0.2010125
MLP Validation Accuracy: 0.1969
MLP Test Accuracy: 0.2042


In [73]:
# calculate Gini impurity, calculate error based on random classification 
def gini_impurity(y):
    # find the counts of each class/label
    class_counts = np.bincount(y.flatten())
    # find the distribution of samples across the classes
    probabilities = class_counts / len(y)
    # calculate the error and return
    return 1 - np.sum(probabilities ** 2)

In [74]:
# finds the best split in the tree
def best_split(X, y, n_features):
    # find the number of features for each password
    num_features = X.shape[1]
    # initialize the minimum impurity (begin with infinity to find smallest impurity)
    min_impurity = float('inf')
    # initialize the specific feature associated with the minimum impurity
    min_impurity_feature = None
    # initialize the minimum impurity feature's corresponding threshold value
    min_impurity_threshold = None

    # choose a random subset of features from all the features
    feature_subset = np.random.choice(num_features, n_features, replace=False)

    # iterate through the features in the subset
    for feature in feature_subset:
        # get the column corresponding to the current feature
        feature_column = X[:, feature]

        # find all unique values in the feature column
        thresholds = np.unique(feature_column)

        # for each threshold value, it is trying to minimize the gini impurity
        for threshold in thresholds:
            # find all the samples whose values in the feature column are less than the threshold
            left_node_samples = y[feature_column < threshold]
            # find all the samples whose values in the feature column are greater than or equal to the threshold
            right_node_samples = y[feature_column >= threshold]

            # if there are samples on either side of the threshold value, then calculate the impurity and update as required
            # essentially filters out invalid thresholds that may be greater/less than all the samples
            if len(left_node_samples) != 0 and len(right_node_samples) != 0:
                # calculate the impurity of the left node samples
                left_impurity = gini_impurity(left_node_samples)
                # calculate the impurity of the right node samples
                right_impurity = gini_impurity(left_node_samples)
                # find the weighted impurity value (expectation)
                weighted_impurity = (len(left_node_samples) * left_impurity + len(right_node_samples) * right_impurity) / len(y)

                # if the weighted impurity is less than the current minimum impurity, then update the values
                if weighted_impurity < min_impurity:
                    min_impurity = weighted_impurity
                    min_impurity_feature = feature
                    min_impurity_threshold = threshold

    return min_impurity_feature, min_impurity_threshold

In [301]:
# builds the decision tree
def build_tree(X, y, max_depth, min_samples_split, depth, n_features):
    # if there are no unique y values or there are fewer samples than required to split the tree 
    # or the depth is greater than the specified max depth, return the majority class of the samples
    if len(np.unique(y)) == 1 or len(y) < min_samples_split or depth >= max_depth:
        return np.argmax(np.bincount(y.flatten()))

    # find the feature and threshold that corresponds to the best split
    feature, threshold = best_split(X, y, n_features)

    # if no feature is returned, return the majority class of the samples
    if feature is None:
        return np.argmax(np.bincount(y.flatten()))

    # get the column corresponding to the feature
    feature_column = X[:, feature]

    # find all the samples whose values in the feature column are less than the threshold
    left_node_samples_X = X[feature_column < threshold]
    left_node_samples_y = y[feature_column < threshold]
    
    # find all the samples whose values in the feature column are greater than or equal to the threshold
    right_node_samples_X = X[feature_column >= threshold]
    right_node_samples_y = y[feature_column >= threshold]

    # recursively build the left and right sides of the tree
    left_tree = build_tree(left_node_samples_X, left_node_samples_y, max_depth, min_samples_split, depth + 1, n_features)
    right_tree = build_tree(right_node_samples_X, right_node_samples_y, max_depth, min_samples_split, depth + 1, n_features)

    return (feature, threshold, left_tree, right_tree)

In [76]:
# classify sample with the decision tree
def predict_tree(x, tree):
    # if the given tree is not a tuple, that means the majority class was returned so return the prediction
    if not isinstance(tree, tuple):
        return tree
    
    # unpack the tuple
    feature, threshold, left_tree, right_tree = tree

    # determine which side of the tree to iterate based on relation of sample's feature value to threshold
    if x[feature] < threshold:
        return predict_tree(x, left_tree)
    else:
        return predict_tree(x, right_tree)

In [77]:
# trains random forest
def random_forest(X, y, n_trees, max_depth, min_samples_split, max_features):
    # initialize list of trees
    trees = []
    # create n number of decision trees
    for tree in range(n_trees):
        # sample with replacement to increase diversity and reduce overfitting
        sample_indices = np.random.choice(len(X), len(X), replace=True)
        # build a decision tree with the randomly chosen samples
        tree = build_tree(X[sample_indices], y[sample_indices], max_depth, min_samples_split, n_features=max_features)
        # store the decision tree
        trees.append(tree)
    return trees

In [346]:
# classify samples with the random forest
def predict_forest(X, trees):
    predictions = []
    # get the prediction from each tree for each sample
    for x in X:
        tree_predictions = np.array([predict_tree(x, tree) for tree in trees])
        predictions.append(tree_predictions)
    # make the list of predictions an array to use bincount
    predictions = np.array(predictions)

    # returns the majority class for each sample as a list
    return [np.argmax(np.bincount(row)) for row in predictions]

In [348]:
trees = random_forest(X_train, y_train, 100, 45, 5, 3)

In [349]:
# find the accuracy metrics for each set of data using Random Forest
# train_predictions = predict_forest(X_train, trees)
# train_accuracy = accuracy(y_train.reshape(1, -1)[0], train_predictions)
# print("Random Forest Train Accuracy:", train_accuracy)

# val_predictions = predict_forest(X_val, trees)
# val_accuracy = accuracy(y_val.reshape(1, -1)[0], val_predictions)
# print("Random Forest Validation Accuracy:", val_accuracy)

test_predictions = predict_forest(X_test, trees)
test_accuracy = accuracy(y_test.reshape(1, -1)[0], test_predictions)
print("Random Forest Test Accuracy:", test_accuracy)

Random Forest Test Accuracy: 0.9423
